In [2]:
# Imports
import os
from dotenv import load_dotenv
import google.generativeai as genai
import requests

print("We are up and running!")

We are up and running!


In [3]:
# Load environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key or api_key.startswith("ADD YOUR"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

print(f"Gemini api key loaded successfully")

Gemini api key loaded successfully


In [4]:
# Configure Gemini
genai.configure(api_key=api_key) # type: ignore
model = genai.GenerativeModel("gemini-2.0-flash") # type: ignore

print(f"Gemini model loaded successfully: {model.model_name}")

Gemini model loaded successfully: models/gemini-2.0-flash


In [5]:
# Ask bitcoin price
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.candidates[0].content.parts)

[text: "I am unable to provide you with the exact, real-time current price of Bitcoin. Cryptocurrency prices are incredibly volatile and change constantly, even within seconds.\n\nHowever, I can tell you the best ways to find the current price:\n\n*   **Reliable Cryptocurrency Exchanges:** Check major exchanges like Coinbase, Binance, Kraken, Gemini, or Bitstamp. They display the current price prominently.\n*   **Financial Websites:** Reputable financial news sites like Yahoo Finance, Google Finance, Bloomberg, and Reuters have cryptocurrency sections with live price trackers.\n*   **Cryptocurrency Tracking Websites:** CoinMarketCap and CoinGecko are dedicated to tracking cryptocurrency prices and have a wealth of other information.\n\nWhen you check, make sure you are looking at the price for the currency you want (e.g., BTC/USD, BTC/EUR, etc.)\n"
]


In [6]:
# Add tool
def get_crypto_price(symbol: str) -> float:
    """
    Get the current price of a cryptocurrency from Binance API
    """
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return float(data["price"])

In [7]:
# Test tool call
price = get_crypto_price("BTCUSDT")
print(f"BTC Price in USDT: {price}")

BTC Price in USDT: 117705.1


In [8]:
# Declare tools
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Get cryptocurrency price in USDT from Binance",
                "parameters": {
                    "type": "object", 
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The cryptocurrency trading pair symbol (e.g., BTCUSDT, ETHUSDT). The symbol for Bitcoin is BTCUSDT. The symbol for Ethereum is ETHUSDT."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

In [9]:
# Ask bitcoin price with tools
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response.candidates[0].content.parts)

[function_call {
  name: "get_crypto_price"
  args {
    fields {
      key: "symbol"
      value {
        string_value: "BTCUSDT"
      }
    }
  }
}
]


In [10]:
# Tool call
price = get_crypto_price("BTCUSDT")

In [11]:
# Share tool call response
final_response = chat.send_message(str(price))
print(final_response.candidates[0].content.parts)

[text: "The current price of Bitcoin is $117,705.1.\n"
]
